In [1]:
import torch
from colpali_engine.models import ColQwen2, ColQwen2Processor
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available


In [2]:
model = ColQwen2.from_pretrained(
    "vidore/colqwen2-v1.0-merged",
    torch_dtype=torch.float32,
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
    ).eval()

In [3]:
processor = ColQwen2Processor.from_pretrained("vidore/colqwen2-v1.0-merged")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
# Your inputs
images = [
        Image.new("RGB", (8192, 8192), color="white"),
        Image.new("RGB", (4096, 4096), color="white"),

        ]
queries = [
        "Is attention really all you need?",
        "What is the amount of bananas farmed in Salvador?",
        ]

In [5]:
# Process the inputs
batch_images = processor.process_images(images, context_prompts=queries)
batch_queries = processor.process_queries(queries)

# Forward pass
with torch.no_grad():
    #     # image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)
    # query_embeddings2 = model(
    #     **batch_queries,
    #     pixel_values=(torch.zeros((2, 0, 0), dtype=torch.float32)),
    #     image_grid_thw=torch.zeros((2, 3), dtype=torch.long)
    #     )
    # query_embeddings2 = model(**batch_queries)  #, pixel_values=None, image_grid_thw= None)

# scores = processor.score_multi_vector(query_embeddings, image_embeddings)

In [6]:
# Create ONNX Runtime session
import onnxruntime as ort

session = ort.InferenceSession("onnx_models/colqwen2_combined.onnx")

In [7]:
# print("\nTesting ONNX model with ONNXRuntime...")

import numpy as np
from colpali_engine.models import ColQwen2, ColQwen2Processor

from converter.colqwen2_model import ColQwen2Wrapper
from converter.dummy_inputs import dummy_inputs_for_export_tuple

og_model = ColQwen2.from_pretrained("vidore/colqwen2-v1.0-merged")
combined_model_wrapper = ColQwen2Wrapper(og_model).eval()
dummy_texts = dict(**batch_queries)
text_input_ids = dummy_texts["input_ids"]
text_attention_mask = dummy_texts["attention_mask"]
text_input_dict = {
        "input_ids": text_input_ids.numpy(),
        "attention_mask": text_attention_mask.numpy(),
        "pixel_values": np.zeros((2, 0, 1176), dtype=np.float32),
        "image_grid_thw": np.zeros((2, 3), dtype=np.int64),
        }

# Convert PyTorch tensors to NumPy arrays for ONNXRuntime
input_dict = {
        "input_ids": dummy_inputs_for_export_tuple[0].numpy(),
        "attention_mask": dummy_inputs_for_export_tuple[1].numpy(),
        "pixel_values": dummy_inputs_for_export_tuple[2].numpy(),
        "image_grid_thw": dummy_inputs_for_export_tuple[3].numpy(),
        }

# Run inference with ONNXRuntime
ort_outputs = session.run(["output_embeddings"], text_input_dict)
print(f"ONNX model output shape: {ort_outputs[0].shape}")

# # # Compare with PyTorch model output
# with torch.no_grad():
#     pytorch_outputs = combined_model_wrapper(
#         **dummy_texts, pixel_values=torch.zeros((2, 0, 1176), dtype=torch.float32),
#         image_grid_thw=torch.zeros((2, 3), dtype=torch.int64)
#         )
#
# mean_diff = np.mean(np.abs(ort_outputs[0] - pytorch_outputs.numpy()))
# print(f"Mean absolute difference between PyTorch and ONNX outputs: {mean_diff}")
# print("ONNX model test complete.")

2025-05-09 14:58:17.741615 [E:onnxruntime:, sequential_executor.cc:572 ExecuteKernel] Non-zero status code returned while running Gather node. Name:'/model/visual/blocks.0/attn/Gather_4' Status Message: indices element out of data bounds, idx=1 must be within the inclusive range [-1,0]


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Non-zero status code returned while running Gather node. Name:'/model/visual/blocks.0/attn/Gather_4' Status Message: indices element out of data bounds, idx=1 must be within the inclusive range [-1,0]

In [7]:
print(input_dict["pixel_values"].shape)
print(text_input_dict["pixel_values"].shape)

(2, 2916, 1176)


KeyError: 'pixel_values'

In [ ]:
input_dict["image_grid_thw"].shape